<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/pinn_training_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch umap-learn numpy

In [ ]:
# pinn_training.py

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import umap
import random

# Reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)


# -----------------------------------------------------------------------------
# 1. Synthetic Data Generators
# -----------------------------------------------------------------------------
def generate_dissolution_data(n_samples=1000):
    """
    dC/dt = -k * C
    Analytical: C(t) = C0 * exp(-k t)
    We generate (t, C(t)) pairs.
    """
    k = 0.5
    C0 = 1.0
    t = np.linspace(0, 10, n_samples)[:, None]
    C = C0 * np.exp(-k * t)
    # Add a small noise
    C += 0.01 * np.random.randn(*C.shape)
    return torch.tensor(t, dtype=torch.float32), torch.tensor(C, dtype=torch.float32)


def generate_mass_accumulate_data(n_samples=1000):
    """
    dM/dt = +k * M
    Analytical: M(t) = M0 * exp(k t)
    We generate (t, M(t)) pairs with noise.
    """
    k = 0.3
    M0 = 0.5
    t = np.linspace(0, 10, n_samples)[:, None]
    M = M0 * np.exp(k * t)
    M += 0.01 * np.random.randn(*M.shape)
    return torch.tensor(t, dtype=torch.float32), torch.tensor(M, dtype=torch.float32)


# -----------------------------------------------------------------------------
# 2. Model Definitions
# -----------------------------------------------------------------------------
class PINN(nn.Module):
    def __init__(self, in_features=1, hidden_dim=64, out_features=1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, out_features),
        )

    def forward(self, x):
        return self.net(x)


# -----------------------------------------------------------------------------
# 3. Physics-Informed Loss Functions
# -----------------------------------------------------------------------------
def physics_residual_dissolution(model, t):
    """
    Residual for dC/dt + k*C = 0
    """
    t.requires_grad_(True)
    C_pred = model(t)
    dC_dt = torch.autograd.grad(C_pred.sum(), t, create_graph=True)[0]
    k = 0.5
    res = dC_dt + k * C_pred
    return res


def physics_residual_accumulate(model, t):
    """
    Residual for dM/dt - k*M = 0
    """
    t.requires_grad_(True)
    M_pred = model(t)
    dM_dt = torch.autograd.grad(M_pred.sum(), t, create_graph=True)[0]
    k = 0.3
    res = dM_dt - k * M_pred
    return res


# -----------------------------------------------------------------------------
# 4. Training Loop
# -----------------------------------------------------------------------------
def train_pinn(model, data_loader, residual_fn, epochs=100, phy_lambda=1.0, lr=1e-3, tag="Model"):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    mse_loss = nn.MSELoss()
    for epoch in range(1, epochs + 1):
        total_data_loss = 0.0
        total_phy_loss = 0.0

        for t_batch, y_batch in data_loader:
            # Zero grad
            optimizer.zero_grad()

            # Data loss
            y_pred = model(t_batch)
            loss_data = mse_loss(y_pred, y_batch)

            # Physics loss (on same batch)
            res = residual_fn(model, t_batch)
            loss_phy = mse_loss(res * 0.0, res)  # target zero residual

            # Combined loss
            loss = loss_data + phy_lambda * loss_phy
            loss.backward()
            optimizer.step()

            total_data_loss += loss_data.item()
            total_phy_loss += loss_phy.item()

        avg_data = total_data_loss / len(data_loader)
        avg_phy = total_phy_loss / len(data_loader)
        print(f"[{tag}] Epoch {epoch:3d}/{epochs} — Data Loss: {avg_data:.6f}, Phy Loss: {avg_phy:.6f}")

    return model


# -----------------------------------------------------------------------------
# 5. Inference & Uncertainty (MC-Dropout)
# -----------------------------------------------------------------------------
def mc_dropout_predict(model, t, n_samples=50):
    model.train()  # keep dropout active
    preds = []
    for _ in range(n_samples):
        preds.append(model(t).detach().cpu().numpy())
    preds = np.stack(preds, axis=0)
    return preds.mean(axis=0), preds.std(axis=0)


# -----------------------------------------------------------------------------
# 6. Main: Orchestrate Everything
# -----------------------------------------------------------------------------
def main():
    # Generate data
    t_diss, C = generate_dissolution_data()
    t_acc, M = generate_mass_accumulate_data()

    # DataLoaders
    batch_size = 128
    loader_diss = DataLoader(TensorDataset(t_diss, C), batch_size=batch_size, shuffle=True)
    loader_acc = DataLoader(TensorDataset(t_acc, M), batch_size=batch_size, shuffle=True)

    # Instantiate models
    model_diss = PINN()
    model_acc = PINN()

    # Train both PINNs
    print("Training DissolutionAI...")
    model_diss = train_pinn(
        model_diss, loader_diss, physics_residual_dissolution,
        epochs=100, phy_lambda=1.0, lr=1e-3, tag="DissolutionAI"
    )

    print("\nTraining PreComputationalAI...")
    model_acc = train_pinn(
        model_acc, loader_acc, physics_residual_accumulate,
        epochs=100, phy_lambda=1.0, lr=1e-3, tag="PreComputationalAI"
    )

    # Inference on full grid
    with torch.no_grad():
        t_full = torch.linspace(0, 10, 500)[:, None]
        mean_diss, std_diss = mc_dropout_predict(model_diss, t_full)
        mean_acc, std_acc = mc_dropout_predict(model_acc, t_full)

    print("\nDissolutionAI Output Mean :", np.array2string(mean_diss.flatten(), precision=6))
    print("DissolutionAI Output Std  :", np.array2string(std_diss.flatten(), precision=6))
    print()
    print("PreComputationalAI Output Mean:", np.array2string(mean_acc.flatten(), precision=6))
    print("PreComputationalAI Output Std :", np.array2string(std_acc.flatten(), precision=6))

    # UMAP projection of latent representations (last hidden layer output)
    # Extract hidden features
    def extract_hidden(model, t):
        x = model.net[0](t)
        x = torch.tanh(x)
        x = model.net[2](x)
        x = torch.tanh(x)
        return x.detach().cpu().numpy()

    feat_diss = extract_hidden(model_diss, t_full)
    feat_acc = extract_hidden(model_acc, t_full)

    reducer = umap.UMAP(n_components=2, random_state=SEED)
    emb = reducer.fit_transform(np.vstack([feat_diss, feat_acc]))
    # emb[:500] = dissolution, emb[500:] = accumulation
    print("\nUMAP embedding computed:", emb.shape)


if __name__ == "__main__":
    main()